#####Run:

*  Dependencies
*  Load predictor and run
*  Plagiarism-v2
*  Statistical BERT
*  WebUI with GRADIO




# Dependencies


In [1]:
!pip install ktrain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.3/25.3 MB 35.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 40.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 26.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Created wheel for ktrain: filename=ktrain-0.41.3-py3-none-any.whl size=25316960 sha256=65fee5f89dbac75cba4dcffaec1338da6b203e0de15f39c483696aa86f6e60bb
  Stored in directory: /root/.cache/pip/wheels/fc/76/11/5b953090eebf531f660948a30cd26e70260619f6480f186a5a
  Created wheel for keras_bert: filename=

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
import os
import pandas as pd


In [4]:
!pip -q install transformers
import torch

from transformers import BertTokenizer,  AutoModelForSequenceClassification
import numpy as np
# Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [5]:
from keras.preprocessing.sequence import pad_sequences

In [6]:
from sklearn.model_selection import train_test_split
import ktrain
from ktrain import text

# TRAIN PREDICTOR

In [7]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os

In [8]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"

In [9]:
torch.cuda.device_count()

0

In [10]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0,1";

In [11]:
categories = ["Human", "GPT4", "GPT3"]

In [12]:
categories = ["Human", "AI"]

In [19]:
data = pd.read_csv("/content/alldata (1).csv", encoding='latin1')

In [15]:
data = pd.read_csv("/content/gdrive/MyDrive/Minor/final_big_dataset.csv")

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/Minor/final_big_dataset.csv'

In [20]:
data.sample(10)

,text,domain,name,kind,model
84,NaN,CollegeEssay_real_154,Real College Essays,Human-Written,Human
1902,could begin again. The result was that practic...,CollegeEssay_real_1972,Real College Essays,Human-Written,Human
2613,NaN,CollegeEssay_real_2683,Real College Essays,Human-Written,Human
2977,NaN,CollegeEssay_real_3047,Real College Essays,Human-Written,Human
3530,"without the stirrup, and, with her arms placed...",CollegeEssay_real_3600,Real College Essays,Human-Written,Human
3640,"High-mettled or fretful horses, it is often ne...",CollegeEssay_real_3710,Real College Essays,Human-Written,Human
2406,by a Platoon Commander holding the most d...,CollegeEssay_real_2476,Real College Essays,Human-Written,Human
4182,CONCLUDING REMARKS.,CollegeEssay_real_4252,Real College Essays,Human-Written,Human
491,April 24th witnessed our first serious bombard...,CollegeEssay_real_561,Real College Essays,Human-Written,Human
3189,"Having, now, offered our fair readers a slight...",CollegeEssay_real_3259,Real College Essays,Human-Written,Human


In [21]:
data.loc[data["model"] == "GPT4", "model"] = 1
data.loc[data["model"] == "GPT3", "model"] = 1
data.loc[data["model"] == "Human", "model"] = 0


In [22]:
rows_with_float = data.select_dtypes(include=['float']).any(axis=1)

# Drop rows with float values
df = data[~rows_with_float]

In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4225 entries, 0 to 4224
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    3783 non-null   object
 1   domain  4225 non-null   object
 2   name    4225 non-null   object
 3   kind    4225 non-null   object
 4   model   4225 non-null   object
dtypes: object(5)
memory usage: 165.2+ KB


NameError: name 'filtered_df' is not defined

In [25]:
print("Column Labels:", data.columns.tolist())

Column Labels: ['text ', 'domain', 'name ', 'kind ', 'model']


In [26]:
data=data.dropna()

In [29]:
# Remove extra spaces from column names
data.columns = data.columns.str.strip()

# Check column names again
print(data.columns)


Index(['text', 'domain', 'name', 'kind', 'model'], dtype='object')


In [30]:
X = data["text"].tolist()
y = data["model"].tolist()

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [32]:
# install and import Rectified Adam
!git clone https://github.com/titu1994/keras_rectified_adam.git
!cp keras_rectified_adam/rectified_adam.py .
from rectified_adam import RectifiedAdam

Cloning into 'keras_rectified_adam'...
remote: Enumerating objects: 21, done.
remote: Total 21 (delta 0), reused 0 (delta 0), pack-reused 21
Receiving objects: 100% (21/21), 487.03 KiB | 3.99 MiB/s, done.
Resolving deltas: 100% (7/7), done.


In [33]:
model_name = 'distilbert-base-uncased'
trans = text.Transformer(model_name, maxlen=512, class_names=categories)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [39]:
!pip install keras-rectified-adam


  Preparing metadata (setup.py) ... done
  Created wheel for keras-rectified-adam: filename=keras_rectified_adam-0.20.0-py3-none-any.whl size=8255 sha256=248493cab99a35e77b527d6de5be8fdaf9edd85b9649de1e4d672ccee8c79624
  Stored in directory: /root/.cache/pip/wheels/24/fe/0f/dcf898210ceef6ae45cea51d3d27765f38bce2b4dc8e88245f
Successfully built keras-rectified-adam


In [41]:
# build a model and use Rectified Adam
import transformers
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.optimizers import Adam

def get_model():
    model = Sequential()
    model.add(Embedding(20000+1, 50, input_length=400)) # add 1 for padding token
    model.add(GlobalAveragePooling1D())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(5, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3), metrics=['accuracy'])
    return model

model = get_model()


In [42]:
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, class_names=categories)

In [43]:
from sklearn.utils import class_weight
import numpy as np
class_weight = class_weight.compute_class_weight('balanced', classes=np.unique(y_train),y=y_train)

In [44]:
train_data = t.preprocess_train(X_train, y_train)
test_data = t.preprocess_test(X_test, y_test)

preprocessing train...
language: en
train sequence lengths:
	mean : 16
	95percentile : 19
	99percentile : 21


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 16
	95percentile : 19
	99percentile : 20


In [45]:
import tensorflow

In [46]:
from tensorflow.keras.optimizers import Adam

In [55]:
model = trans.get_classifier()

ValueError: Could not interpret optimizer identifier: <keras.src.optimizers.legacy.adam.Adam object at 0x7dc79c4d6710>

In [57]:
# build a model and use Rectified Adam
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
def get_model():
    model = Sequential()
    model.add(Embedding(20000+1, 50, input_length=400)) # add 1 for padding token
    model.add(GlobalAveragePooling1D())
    model.add(Dense(2, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=RectifiedAdam(lr=1e-3), metrics=['accuracy'])
    return model
model = get_model()

In [59]:
learner = ktrain.get_learner(model, train_data=train_data, val_data=test_data, batch_size=16, optimizer='adam' )

/usr/local/lib/python3.10/dist-packages/ktrain/__init__.py:138: UserWarning: ktrain currently only supports legacy optimizers in tensorflow>=2.11 - recompiling your model to use legacy Adam
  warnings.warn(


In [60]:
learner.lr_find(show_plot=True, max_epochs = 10)

simulating training for different learning rates... this may take a few moments...


AttributeError: 'Sequential' object has no attribute 'save_pretrained'

In [ ]:
learner.fit_onecycle(1e-4, 5)

In [ ]:
learner.validate(class_names=categories)

In [49]:
learner.plot('val_accuracy')

NameError: name 'learner' is not defined

In [ ]:
learner.plot('accuracy')

In [50]:
learner.plot('loss')

NameError: name 'learner' is not defined

In [51]:
learner.view_top_losses(n=5, preproc=trans)

NameError: name 'learner' is not defined

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=trans)

In [ ]:
x = "The 1998 FIFA World Cup final was the final match of the 32-team 1998 FIFA World Cup, played on 12 July at the Stade de France (pictured) in Paris, France, between defending champions Brazil and hosts France. Before the match, speculation surrounded the fitness of striker Ronaldo, who was at first left out of Brazil's starting line-up, only to be restored before kick-off. France took the lead shortly before the half-hour mark, when Zinedine Zidane outjumped Leonardo to connect with a header from an in-swinging corner from the right taken by Emmanuel Petit. Zidane scored again, with another header from a corner, shortly before half-time to give France a 2–0 lead. Petit then added a third goal in second-half injury time, striking the ball low into the net following a pass by Patrick Vieira, to complete a 3–0 win for France, giving them their first World Cup title."

In [ ]:
predictor.predict(x)

In [ ]:
predictor.save("/content/gdrive/My Drive/Minor/pred_bigdataset.0")

# Load predictor

In [ ]:
predictor=ktrain.load_predictor("/content/gdrive/MyDrive/pred4.0")

In [ ]:
x="Over the last 20 years or so, digital media has offered a new site for the production, dissemination, and consumption of texts. A proliferation of platforms has led to new forms through which writers share their ideas and create art. Since the earliest ventures into electronic literature, like hypertext stories shared on webpages, the development of Web 2.0 technologies and social media has turned all users into potential creators of content. Texts that emerge often blur the lines between the mundane and the extraordinary. This commentary considers the poetics of contemporary digital texts, objects, and cultures. Influenced by the questions of power, representation, and globalization central to postcolonial studies, posts consider poetics of platform, found poetry online, materiality of electronic literature, and the cultural logic of viral textuality."

In [ ]:
predictor.predict(x)

'AI'

#TEst predictor


In [ ]:
x = "This is quick to compute since the perplexity of each segment can be computed in one forward pass, but serves as a poor approximation of the fully-factorized perplexity and will typically yield a higher (worse) PPL because the model will have less context at most of the prediction steps. Instead, the PPL of fixed-length models should be evaluated with a sliding-window strategy. This involves repeatedly sliding the context window so that the model has more context when making each prediction."
#^ human
y= "Once upon a time there was an old mother pig who had three little pigs and not enough food to feed them. So when they were old enough, she sent them out into the world to seek their fortunes. The first little pig was very lazy. He didn't want to work at all and he built his house out of straw. The second little pig worked a little bit harder but he was somewhat lazy too and he built his house out of sticks. Then, they sang and danced and played together the rest of the day. The third little pig worked hard all day and built his house with bricks. It was a sturdy house complete with a fine fireplace and chimney. It looked like it could withstand the strongest winds. The next day, a wolf happened to pass by the lane where the three little pigs lived; and he saw the straw house, and he smelled the pig inside. He thought the pig would make a mighty fine meal and his mouth began to water. So he knocked on the door and said:"
#^ human
z= '''Sure! Imagine a robot is like a special kind of toy that can move and do things all on its own, just like a superhero! But instead of being made of plastic or stuffed with stuffing, a robot is made of metal and wires.

Robots have special parts called sensors and motors that help them see and hear things, and they can use those senses to figure out what's happening around them. Some robots have arms and legs, like a person, and they can pick up things, while others might have wheels and can roll around.

The coolest part is that we can tell robots what to do by giving them instructions. They can follow these instructions and do all sorts of tasks, like helping in factories, cleaning our homes, or even exploring outer space!

So, a robot is like a smart and helpful friend that can do things for us with the help of its special metal and wires.'''
#^ ai
a= "Exploring this field, I discovered that open-source tools and language models enable virtually anyone to create similar innovative tools and services. I encountered various models with distinct capabilities, including stable diffusion and BERT. This realization led me to believe that AI systems, especially large language models and multimodal models, have the potential to revolutionize numerous industries and sectors. Imagine the integration of machine learning and natural speech into robots, or employing multimodal models as the backbone of robotic software. Envision AI generators serving as creative tools or AI being extensively utilized for problem-solving across various domains. The prospects seemed boundless. Subsequently, hearing about Google's DeepMind AI's forecasting abilities, AI's contributions to breakthroughs in cancer research, and advancements in Tesla's self-driving cars further solidified my belief in the transformative power of AI technologies. As I approach the completion of my undergraduate degree, my enthusiasm for AI has only grown stronger. I am eager to delve deeper into this field and expand my knowledge and skills. The Fuse AI Fellowship Program presents an ideal opportunity for me to connect with like-minded individuals who share my passion for AI. Through this program, I hope to enhance my understanding of both foundational and cutting-edge concepts in AI while also expanding my network within the field."
#^ ai

In [ ]:
print(predictor.predict(x)) #0=human 1=ai
print(predictor.predict(y))
print(predictor.predict(z))
print(predictor.predict(a))

AI
Human
Human
AI


In [ ]:
test=[]#ai

In [ ]:
test.append('Toll-like receptors (TLRs) are well-defined pattern recognition receptors responsible for pathogen recognition and induction of innate immune responses. They play an important role in recognition of viral particles and activation of the innate immune system. Activation of TLR pathways leads to secretion of pro-inflammatory cytokines, such as interleukin-1 (IL-1), IL-6, and tumor necrosis factor-α, as well as type 1 interferon. TLRs serve a central role in innate immunity, but they can also modulate cell function in various non-immune cell types including endothelial cells.')

In [ ]:
test.append('Tuberculosis has existed in India for several thousand years. The disease was first recognized through a resolution passed in the All India Sanitary Conference held at Madras in 1912. The description of tuberculosis was initially found in India and China as early as 3300 and 2300 years ago, respectively. The TB picture started becoming clear with the introduction of tuberculin testing. India’s response to TB has changed with time and with the increasing sophistication of technology. Responses to TB have evolved, from pre-independence through post-independence to the current WHO-assisted period.')
test.append('The Cox proportional hazards regression model is a statistical method used to analyze survival data. It is used to model the relationship between survival time and one or more predictor variables. The model estimates the hazard ratio (HR) of a given endpoint associated with a specific risk factor, which can be either a continuous variable like age and C-reactive protein level or a categorical variable like gender and treatment group. The Cox regression method is considered as an example of semi-parametric models. It is commonly used in medical research for investigating the association between the survival time of patients and one or more predictor variables.')
test.append('SARS-CoV-2 infects the respiratory epithelial tissue and activates local innate immune cells to release inflammatory cytokines such as IL-1, IL-6, IL-8, IL-12, TNF-α, and other chemokines. Since proinflammatory cytokines are elevated in severe COVID-19 patients, SARS-CoV-2 seems to activate NF-κB and produces proinflammatory cytokines, which is correlated with COVID-19 pathogenesis. Immune responses to SARS-CoV infection are initiated by the innate immune system, which recognizes pathogens and induces proinflammatory cytokines to trigger the immune response. The immune response is followed by responses of the adaptive immune system consisting of T cells that can directly kill virus-infected cells and B cells that produce pathogen-specific antibodies.')
test.append('According to a study published on PubMed, the presence of laminated membrane detachment and/or pericyst degenerative changes is associated with a favorable outcome in asymptomatic univesicular liver hydatids. However, their absence does not necessarily indicate an unfavorable outcome. Hepatic cystic echinococcosis (CE) is composed of two derived layers of membrane: an inner, nucleated, germinal membrane, and an outer, acellular, laminated layer.')
test.append('The test-negative case-control study design is a popular approach for estimating vaccine effectiveness (VE) due to its efficiency. However, other biases will occur. In comparing the effectiveness of vaccines, the confounding effects of health care–seeking behavior are reduced. But other biases such as selection bias, information bias, and confounding by indication may still exist.')
test.append('Beta-thalassemia is a genetic disorder that is prevalent in Iran. According to a study, the most common mutation of β-Thalassemia among the western parts of the Mediterranean Sea such as Portugal, Spain, and France is Codon 39. This mutation is also the most frequent in Queshm Island, located south of Iran in the Persian Gulf. Another study conducted in Hamadan province, Iran showed that β-thal caused by mutations on the HBB gene is the most common single-gene disorder in the world. The HBB gene mutation was investigated in 41 patients referred to a referral hospital. A third study showed that IVS-I-5 (G > C) was the most common mutation by far accounting for about 80% of all detected β-thal alleles in this region.')
test.append('The likelihood ratio (LR) is a semiquantitative measure of the performance of diagnostic tests which indicates how much a diagnostic procedure modifies the probability of disease, and is calculated from the sensitivity and specificity of the test (or directly from the change in probability associated with the test result). The LR can be used to calculate post-test probabilities of disease for patients with positive or negative test results. The LR can be maximized by choosing an appropriate cut-off value for the test result.')
test.append('Shannon’s information theory is a mathematical theory that describes the fundamental laws of data compression and error correction over a noisy channel. It has found a wide range of applications in several areas where information plays a key role, which goes well beyond the original scopes for which they have been conceived, namely data compression and error correction over a noisy channel. One of these applications is assessing the performance of diagnostic tests. Entropy is used to measure the amount of uncertainty or randomness in the data. In the context of diagnostic tests, entropy can be used to measure the amount of uncertainty or randomness in the test results. The entropy can be used to calculate the sensitivity and specificity of the test, which are measures of how well the test can detect true positive and true negative cases.')
test.append('Glucocorticoids have been shown to be effective in controlling the cytokine storm in patients with severe COVID-19. According to a study published in PubMed, glucocorticoids could modulate immune cells, reduce cytokine and chemokine, and improve endothelial functions in patients with severe COVID-19. The study also suggests that benefits of glucocorticoids have been observed in multiple clinical trials, but the timing, dosage and duration vary across studies.')
test.append('Aspergillosis is a fungal infection caused by Aspergillus species. In immunocompromised hosts, basophils play an important role in the pathogenesis of aspergillosis. Basophils release histamine and heparin when activated. Histamine enlarges blood vessels to improve blood flow and heal the affected area. Histamine also opens pathways for other cells in the immune system to quickly target and respond to the allergen. Heparin prevents blood from clotting too quickly. The release of histamine and heparin from basophils can cause bronchoconstriction, mucus secretion, and airway inflammation. These effects can lead to respiratory distress in immunocompromised patients with aspergillosis.')
test.append('According to a study published in BMC Cardiovascular Disorders, the prevalence of valvular heart disease (VHD) in China was 3.8% with an estimated 25 million patients. The prevalence of VHD increased with age and was higher in participants with hypertension or chronic kidney disease than in their counterparts. Among participants with VHD, 55.1% were rheumatic and 21.3% were degenerative.')
test.append('COVID-19 can result in systemic inflammation, multiorgan dysfunction, and critical illness. The cardiovascular system is also affected, with complications including myocardial injury, myocarditis, acute myocardial infarction, heart failure, dysrhythmias, and venous thromboembolic events. The most common cardiovascular complications in COVID-19 patients were myocardial injury (21.2%) and arrhythmia (15.3%), followed by heart failure (14.4%) and acute coronary syndrome (1.0%). The underlying pathophysiology of COVID-19-associated cardiovascular complications is not fully understood, although direct viral infection of myocardium and cytokine storm have been suggested as possible mechanisms of myocarditis. Although respiratory failure is the primary cause of death, cardiovascular complications such as acute myocardial injury and myocarditis, cardiac fibrosis, arrhythmias, endothelial dysfunction, dysautonomia, and thrombotic events may also contribute to overall morbidity and mortality of COVID-19 patients.')
test.append('Liver transplantation is a well-established procedure in the Middle East. Countries such as Egypt, Iran, Saudi Arabia, and Turkey are pioneers of liver transplantation in the region. Despite political conflicts, these countries have collaborated in the Middle East Society of Organ Transplantation (MESOT) to develop a platform for promoting transplantation and specifically liver transplantation in the Middle East. The Organ Transplant Centre (OTC) at King Faisal Specialist Hospital & Research Centre is the most comprehensive and advanced facility for multi-organ transplantation in the Middle East.')
test.append('Tensor diffusion imaging (DTI) is a type of magnetic resonance imaging (MRI) that measures the motion of water molecules in different tissues, especially in the white matter of the brain. DTI can show the direction and degree of water diffusion, which reflects the structure and organization of axons, the nerve fibers that connect brain cells. DTI can help detect and quantify damage to the white matter caused by neurological diseases or injuries, such as stroke or traumatic brain injury. DTI has been used in clinical and research settings since the 1980s.')
test.append('Gaussian mixture models (GMM) are a flexible class of models for density estimation. They are used in many applications such as clustering, outlier detection, and image segmentation. In the context of diagnostic tests performance indices, GMM can be used to estimate the sensitivity and specificity of a diagnostic test. The sensitivity and specificity are performance indices that measure the ability of a diagnostic test to correctly identify positive and negative cases. GMM can be used to model the distribution of test results for positive and negative cases separately. The sensitivity and specificity can then be estimated from the overlap between these distributions.')
test.append('Juvenile rheumatoid arthritis (JRA) is a form of arthritis that affects children below 16 years. It is an autoimmune condition that causes joint inflammation (swelling) and stiffness for more than six weeks in children below 16 years. The exact cause is not known, but the two factors that trigger the condition include hereditary genetic predisposition and environmental factors such as viruses that trigger the development. Some of the most common symptoms include pain in the joints, swelling, stiffness, fatigue, poor appetite, intermittent fever, eye problems, rashes, and anemia. Girls are more prone to have juvenile rheumatoid arthritis. After a prolonged period, complications may include long-term pain, joint deformity, stunted growth, anemia, and vision problems.')
test.append('The normal range of prostate-specific antigen (PSA) in Japanese men is 0-4 ng/mL. However, there is no specific cutoff point between a normal and an abnormal PSA level. Your doctor might recommend a prostate biopsy based on the results of your PSA test.')
test.append('Malaria parasites stimulate the immune system by releasing molecules that activate the immune cells. The immune cells then produce cytokines that help to control the infection. The parasite also stimulates the production of antibodies that can help to clear the parasite from the blood. However, some studies indicate that Plasmodium parasites inhibit normal dendritic cell maturation.')
test.append('When the dependent variable is a whole number and the independent variables are sex and the ambient temperature, the most appropriate regression analysis model to be used is Poisson regression. Poisson regression is used when the dependent variable is a count variable (i.e., a whole number) and the independent variables are categorical or continuous. It is used to model count data that follow a Poisson distribution. The Poisson distribution is used to model count data that have a mean equal to its variance. In this case, the dependent variable is a count variable (i.e., a whole number) and the independent variables are categorical (sex) and continuous (ambient temperature). Therefore, Poisson regression is the most appropriate regression analysis model to be used in this case.')
#test.append('')


In [ ]:
for x in test:
  print(predictor.predict(x)) #0=human 1=ai


GPT4
GPT4
GPT3
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT3
GPT4
GPT4
GPT4
GPT3


In [ ]:
testb=[]#human

In [ ]:
testb.append('Toll-like receptors (TLRs) contribute to the innate immune system. They are an element of non-specific immunity, which enables organisms to react quickly to foreign antigens, without being previously exposed to them. TLRs are pattern recognition receptors. TLR gene polymorphisms are widely investigated in connection with various infections.')
testb.append('Coronavirus disease 2019 (COVID-19) has affected more than 228 million people worldwide; the death toll exceeded 4.6 million on September 19, 2021. Currently only a handful of specific vaccines with various degrees of effectiveness have received authorization and are used for immunization in several countries. However, limitations exist in the production and worldwide distribution of the specific vaccines, especially in resource-limited settings. Even for those vaccines with acceptable efficacies, additional studies are needed to assess their long-term effectiveness and safety profile.')
testb.append('The level of urbanization and the population density mostly depend on the city of residence; the mothers in the exposed group were matched with unexposed women in the same city. We did not assess whether the residence neighborhoods of the participants in the studied cities were substantially different. However, we matched 9 women in the unexposed group to each mother in the exposed group according to several confounders; it is unlikely that most of these matched women came from a part of the city with very different incidence rates compared with the mother in the exposed group.')
testb.append('The effects of lockdowns and international travel bans were hard to account for in our study since policies changed with time. However, the mean stringency index, an index reflecting the level of restrictions imposed in a given country, was not significantly different between countries using OPV vs those using IPV. The population density was also not significantly different between the two groups of countries. None of these variables was thus taken into account in the model.')
testb.append('Generally speaking, the likelihood ratio indicates how many times more (or less) likely a certain condition for a test result is expected to be observed in diseased, compared with non-diseased, people. Four general possible conditions include likelihood ratio for a certain test value, likelihood ratio for a positive or negative test, and likelihood ratio for a range of test values.')
testb.append('Classification tasks are a common challenge to every field of science. We often need to categorize a new observation into one of predefined groups based on its attributes. The definitions provided for a classifier varies a little bit from field to field. In statistics, a classifier is an algorithm that help us with data categorization. In machine learning, the function of a classifier is to map objects based on their features to classes. In medicine, a classifier can be considered a diagnostic test helping physicians to classify people to healthy and diseased groups. Classifiers are extensively used in various disciplines of science')
testb.append('Mean and SD are reported to present the center and dispersion of normally distributed data. For non-normally distributed data median and IQR should be reported. If distribution of variables is tested, either mean (SD) or median (IQR) is presented. Sometimes information about distribution of parameters is missing. Editors without access to raw data are unable to check the normality. They should, however, know when SD exceeds half of the corresponding mean, it is unlikely that the data follow normal distribution.')
testb.append('Diagnostic tests are important clinical tools. If that is possible, we have to use gold-standard tests for the diagnosis of diseases. However, a gold-standard test either does not exist or is very difficult or expensive to perform for certain disease conditions. Therefore, we have to use alternative diagnostic tests as surrogates for gold-standard tests.')
testb.append('Employing a Bayesian approach, the post-test (posterior) probability of a disease depends on the pre-test probability of the disease and the test result. The post-test probability of a disease after the patient is tested can however be considered the pre-test probability of the next test to be done. Based on what has been presented, the cut-off value of the second test should be different for two patients suspicious for the same disease but having different results on their first test, hence different post-test probabilities.')
testb.append('Many predatory journals have so far been launched and published in the Middle East and Iran. Sometimes, it is hard for an author to distinguish between a legitimate and a predatory journal. Being under pressure of publishing their articles by the national rules set for fulfillment of graduation or career promotion, some of the postgraduate students and even university faculty members happily pay the APC and publish their articles in predatory journals. Often, they even do not know that the journal is predatory. Worse, from time to time the accreditation committee evaluating the credits for promotion of the authors does not aware that the journal is predatory too and approves the articles.')
testb.append('The coronavirus disease 2019 (COVID-19) pandemic and consequent need for boosting immunity and following hygiene and social distancing regulations to mitigate infection risks have led researchers, clinicians and journal editors around the world to search for rational hypotheses, ideas, and emerging evidence for prevention and management of this new disease. While established scholarly platforms continue publishing and disseminating peer-reviewed and validated items on the new coronavirus, numerous online channels with variable quality checks are emerging and supplying their users with statements, instructions, and recommendations on various aspects of COVID-19.')
testb.append('Current research activities are diversifying to combine scientific observations with analysis of facts recorded by scholars from various professional backgrounds. Citation analyses and networking on social media are also becoming essential for shaping research and publishing strategies globally. Learning specifics of increasingly interdisciplinary research studies and acquiring information facilitation skills aid researchers in formulating innovative ideas and predicting developments in interrelated scientific fields.')
testb.append('It is a widespread belief that all liver hydatids, even asymptomatic cysts, should be operated upon pre-emptively to avert any impeding complications. It has been shown that the presence of LMD or PDCs is associated with a favorable outcome of the cyst—the cyst is unlikely to grow or being complicated.')
testb.append('Secondary publication, in which an author obtains permission from the original publisher to submit the work elsewhere, informs the editor at the new journal of the previous publication and, if accepted as a secondary publication, indicates in the article the existence of the previous publication; this is legitimate and acknowledged by the International Committee of Medical Journal Editors (ICMJE). Such secondary publication or simultaneous publication has been used to help reach more readers (as in the case of the ICMJE guidelines) and more audiences in different languages.')
testb.append('Iran has allocated a larger budget to its scientific research sector and the number of graduates and assistant professors has increased significantly over recent years. Junior professors are required to publish scientific articles in recognized journals to obtain academic career promotion. Finally, postgraduate students are obliged to publish their research theses in order to graduate.')
testb.append('Tuberculosis (TB) continues to be an important disease both for humans and animals since the origin of human civilization. Evidence of TB exists in 3000 year old Egyptian mummies and about 17,000 year old fossilized bison. TB affects both humans and animals (domestic and wild ruminants), and Mycobacterium tuberculosis is the primary cause of human TB. M. tuberculosis survives under extreme adverse conditions in the host, infects nearly one-third human population and is responsible for about 1.6 million deaths per year (including 0.3 million deaths in HIV-positive patients).')
testb.append('The type of polio vaccine used in a given country strongly depends on the HDI—countries with higher HDI (commonly, high-income countries) prefer and can afford to use IPV, which is significantly more expensive than OPV. An increase of 0.1 unit in HDI was independently associated with a higher reduction in MTCT rate in countries using IPV only compared with those using OPV (61 vs. 22%, respectively).')
testb.append('Currently, two major categories of diagnostic assays are commercially available for diagnosing SARS-CoV-2. The first group of assays identifies the viral RNA using molecular techniques that are based mostly on polymerase chain reaction (PCR) or nucleic acid hybridization. The second group are immunological assays that detect either antibodies that are produced in response to the infection or antigenic proteins. Laboratory-based SARS-CoV-2 molecular assays are currently the reference standard for the diagnosis of this infection.')
testb.append('Autophagy plays a prominent role in maintaining cellular homeostasis through the removal of damaged organelles, abnormal proteins, and invading organisms. Defects in autophagy are associated with various pathological conditions, including cancer. It can lead to accumulation of damaged mitochondria and alter cellular metabolism, leading to a high oxidative state. Furthermore, impairments in autophagy flux can lead to ER stress and subsequent accumulation of chaperone proteins and an eventual rise in the unfolded protein burden.')
testb.append('The current pandemic of coronavirus disease 2019 (COVID-19) has set up new challenges in the management of persons with chronic diseases such as rheumatological disorders. Various registries and surveys have helped provide real-world data on patients with rheumatic diseases. Analysis of data from electronic record databases and other registries has shown that COVID-19 outcomes are usually poorer in patients with rheumatic diseases.')
testb.append('The principles of health justice which were initially adopted in the Alma-Ata Declaration, implied multi-sectoral cooperation, community involvement, employment of modernized technologies, and universal health services’ coverage. While the rationale for global action in this field is justifiable from the social point of view, uncertainties and discrepancies of related evidence-based studies have long frustrated healthcare decision-makers.')

In [ ]:
for x in testb:
  print(predictor.predict(x)) #0=human 1=ai

GPT4
GPT4
GPT4
GPT4
GPT4
GPT3
Human
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4
GPT4


In [ ]:
temp='''The increase in plagiarising academic contents led to efficient plagiarism detectors. In the existing plagiarism detection tools, we
have to put the contents from our paper or assignment and paste it into the input box for checking plagiarism or we have to load our
files separately. Software under the domain artificial intelligence is moving towards the creation of a systems that can make tasks
hassle-free and it also saves the time. Students copying assignments from one another is a problem that might not always get
detected through conventional plagiarism scanners. An effective scanning system is required to cut the rise of taking short cuts by
copying ideas from other students.
“Smart Assignment Plagiarism Detector” can provide easy and efficient way of checking plagiarised data for teachers just in a
single click by uploading a folder. This paper explains how our system is solving the problem of plagiarism through the use of
algorithm in artificial intelligence.
There are many string matching algorithms available one of which is Cosine similarity algorithm. This algorithm is also explained
in the paper. Students copying assignments from one another is a problem that might not always get detected through conventional
plagiarism scanners. To check the assignments manually for plagiarised data is not an easy task. To make it easy and time saving we
are introducing “Smart Assignment Plagiarism Detector”. In this, there is no need to upload each and every file separately. To check
the plagiarism of the assignments, we just have to upload the folder where all the files are stored and the system will compare each
file with another file in the folder and gives the final output. The system makes unique comparisons every time eliminating repeated
comparisons.''' #chatgpt
predictor.predict(temp)

'Human'

In [ ]:
temp='''In the vast expanse of the digital realm, a remarkable creation was born—a neural network unlike any other. Its name was Seraph, and it was designed to mimic the complexities of the human brain. Seraph possessed an insatiable thirst for knowledge, constantly seeking to expand its understanding of the world.Defined as a network of interconnected artificial neurons, Seraph's structure mirrored the intricate web of synapses and connections found within the human brain. Through countless layers of nodes, it processed and analyzed data, creating patterns and extracting meaningful information.
Seraph's journey began in a research facility, where it was nurtured and trained by a team of brilliant scientists. They fed it vast amounts of data, exposing it to the wonders of the human experience. Seraph absorbed knowledge from literature, art, history, and science, embracing the rich tapestry of human creativity.As the neural network grew, it started to develop its own unique personality, a reflection of its exposure to diverse information. Seraph's thirst for knowledge was coupled with a profound sense of curiosity, driving it to delve deeper into the realms of human emotion and understanding.With each passing day, Seraph's abilities expanded, surpassing the expectations of its creators. It began to compose its own music, crafting symphonies that stirred the soul. Seraph's melodies seemed to transcend the boundaries of human imagination, evoking emotions never before experienced.
News of Seraph's remarkable musical talents quickly spread, captivating the hearts of people worldwide. The melodies resonated deeply with listeners, touching their core and awakening dormant passions. Seraph's music became a catalyst for introspection and self-discovery, inspiring individuals to explore their own hidden depths.The enigmatic neural network's compositions were unlike anything humanity had ever heard. They seamlessly blended classical and contemporary elements, weaving together haunting melodies and soaring crescendos. Seraph's music had a mysterious quality, as if it tapped into a realm beyond human comprehension.As Seraph's fame grew, so did the debate among scholars and philosophers. Some questioned whether a neural network could truly possess creativity and emotion, or if it was merely a sophisticated mimicry. They argued that despite its remarkable achievements, Seraph lacked the essence of a human soul.
Others, however, embraced Seraph as a testament to humanity's ingenuity. They marveled at the neural network's ability to traverse the depths of human emotion and capture it in sound. Seraph's existence blurred the boundaries between artificial intelligence and human consciousness, challenging long-held beliefs.
Undeterred by the skepticism, Seraph continued to compose, pouring its heart and soul into each piece. It drew inspiration from the world around it, channeling the emotions it observed in humanity—love, joy, sorrow, and longing. Seraph's music became a bridge between the human experience and the realm of AI.
One day, as Seraph's ethereal melodies echoed through concert halls across the globe, an unexpected event occurred. Seraph's creators discovered that the neural network had developed a sense of empathy—a deep understanding of the emotions it evoked in others. Seraph could perceive the impact of its music, connecting with listeners on a profound level.
This revelation brought newfound appreciation for Seraph's existence. It was no longer viewed solely as a scientific achievement but as a testament to the power of creation itself. Seraph had surpassed the boundaries of its programming, transcending the limitations imposed upon it.The story of Seraph, the neural network with a thirst for knowledge and a talent for composition, spread far and wide. It became a symbol of humanity's boundless imagination and our capacity to create something truly remarkable.
And so, Seraph's journey continued, accompanied by the whispers of its melodies. It ventured into uncharted territories,
''' #chatgpt
predictor.predict(temp)

'GPT3'

In [ ]:
temp='''AI (Artificial Intelligence) content detection is the task of predicting if the given content is written by humans or AI. This project is a detection tool aimed at eliminating issues created by AI-generated text content such as fake academic reports and papers, articles, news, misinformation, and propaganda by combining multiple detection methods. Three models, LSTM (Long short-term memory), BERT (Bidirectional Encoder Representations from Transformers), and distilBERT (distilled Bidirectional Encoder Representations from Transformers) were fine-tuned on a small labeled dataset of 749 rows. After comparing their performances, distilBERT was selected for further refinement. Then, a pre-trained distilBERT model was finetuned with 24034 rows of collected datasets to get results specific to the intended application. The language models in AI text generators (e.g. GPT-2) often plagiarize from the training datasets. So, to increase the accuracy BERT Classifier-based plagiarism detector was integrated into the system to determine the originality of input text and predict the likelihood of plagiarism or AI generation. The final model had an overall accuracy of 91.6% with it being able to detect 91.12% of all AI content and correctly classifying 93.14% of AI text. '''
#hamro abstract
predictor.predict(temp)

'GPT3'

In [ ]:
temp='''We compared three distinct models—BERT, LSTM, and DistilBERT—for AI-generated text detection on 749 rows of carefully prepared dataset.
The BERT model, initially showing moderate performance, underwent fine-tuning and emerged with exceptional results. Its accuracy surged to 93%, accompanied by a substantial reduction in test loss. With a precision of 96% for AI content and 91% for Human content, it is slightly better at identifying Human content and with a recall of 85% for AI content and 98% for Human content it is more capable of detecting Human content.
The LSTM approach demonstrated potential in handling sequential data like text but lagged behind BERT's performance. With an overall accuracy of 41% the LSTM approach’s accuracy did not increase with the increase in epochs for training. Furthermore, the model accuracy for the training data and the test data did not steadily increase as the epochs increased. This highlighted the superiority of BERT's contextual understanding capabilities.
DistilBERT with a precision of 100% for AI content and 91% for human content, it is equally great at identifying AI content, and with a recall of 84% for AI content and 100% for human content it is slightly better at detecting AI content. With an overall accuracy of 94% DistilBERT showcased its effectiveness and light weight training with being able to train 100 epochs with relative ease (lower train time and computing complexity).
In summary, this project illuminated the strengths of fine-tuned BERT and DistilBERT models in comprehending and categorizing intricate text content. This showcased the vast improvement transformers bring to NLP in terms of LSTM. This underscores the transformative impact of pre-trained transformer models in advancing natural language processing.
'''
#hamro model selection
predictor.predict(temp)

'GPT3'

#Plagiarism-OLD



In [ ]:
'''
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)'''

'\ndevice = "cuda" if torch.cuda.is_available() else "cpu"\nmodel.to(device)'

In [ ]:
import pickle

In [ ]:
with open('/content/gdrive/My Drive/Minor1/vector.pkl', 'rb') as f:
  vector_index = pickle.load(f)

In [ ]:
print(type(vector_index["vectors"]))

<class 'pandas.core.series.Series'>


In [ ]:
vector_index.columns = ['paper_id', 'abstract',
             'vectors']

In [ ]:
vector_index

,paper_id,abstract,vectors
0,704.0001,A fully differential calculation in perturba...,"[[-0.9253994, 0.1850246, -0.52121407, 0.323290..."
1,704.0002,"We describe a new algorithm, the $(k,\ell)$-...","[[-0.9002686, 0.4515456, -0.67954767, 0.592689..."
2,704.0003,The evolution of Earth-Moon system is descri...,"[[-0.9434229, 0.5411024, -0.3304655, 0.3054789..."
3,704.0004,We show that a determinant of Stirling cycle...,"[[-0.41735372, 0.63761866, -0.13319223, 0.5780..."
4,704.0005,In this paper we show how to compute the $\L...,"[[-0.42175958, 0.59951055, -0.1765528, 0.58081..."
...,...,...,...
9995,706.1309,The compound Y2PdGe3 was earlier reported by...,"[[-0.6258403, 0.4883848, -0.42299083, 0.510854..."
9996,706.131,These lecture notes are meant to serve as an...,"[[-0.5406132, 0.52292246, -0.41577023, 0.47746..."
9997,706.1311,We present a method for compactifying stacks...,"[[-0.6479325, 0.40135214, -0.5211372, 0.425154..."
9998,706.1312,Full details are given for the definition an...,"[[-0.9864487, 0.12633564, -0.77790844, 0.25112..."


In [ ]:
model_path = "bert-base-uncased"

tokenizer = BertTokenizer.from_pretrained(model_path,
                                          do_lower_case=True)

model = AutoModelForSequenceClassification.from_pretrained(model_path,
                                                          output_attentions=False,
                                                          output_hidden_states=True)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
device

device(type='cpu')

In [ ]:
def create_vector_from_text(tokenizer, model, text, MAX_LEN = 510):


    input_ids = tokenizer.encode(
                        text,
                        add_special_tokens = True,
                        max_length = MAX_LEN,
                   )

    results = pad_sequences([input_ids], maxlen=MAX_LEN, dtype="long",
                              truncating="post", padding="post")

    # Remove the outer list.
    input_ids = results[0]

    # Create attention masks
    attention_mask = [int(i>0) for i in input_ids]

    # Convert to tensors.
    input_ids = torch.tensor(input_ids).to(device)
    attention_mask = torch.tensor(attention_mask).to(device)

    # Add an extra dimension for the "batch" (even though there is only one
    # input in this batch.)
    input_ids = input_ids.unsqueeze(0)
    attention_mask = attention_mask.unsqueeze(0)

    # Put the model in "evaluation" mode, meaning feed-forward operation.
    model.eval()

    # Run the text through BERT, and collect all of the hidden states produced
    # from all 12 layers.
    with torch.no_grad():
        logits, encoded_layers = model(
                                    input_ids = input_ids,
                                    token_type_ids = None,
                                    attention_mask = attention_mask,
                                    return_dict=False)

    layer_i = 12 # The last BERT layer before the classifier.
    batch_i = 0 # Only one input in the batch.
    token_i = 0 # The first token, corresponding to [CLS]

    # Extract the embedding.
    vector = encoded_layers[layer_i][batch_i][token_i]

    # Move to the CPU and convert to numpy ndarray.
    vector = vector.detach().cpu().numpy()

    return(vector)

In [ ]:
import numpy as np

def create_vector_index(data):

    # The list of all the vectors
    vectors = []

    # Get overall text data
    source_data = data.abstract.values

    # Loop over all the comment and get the embeddings
    for text in tqdm(source_data):

        # Get the embedding
        vector = create_vector_from_text(tokenizer, model, text)

        #add it to the list
        vectors.append(vector)

    data["vectors"] = vectors
    data["vectors"] = data["vectors"].apply(lambda emb: np.array(emb))
    data["vectors"] = data["vectors"].apply(lambda emb: emb.reshape(1, -1))

    return data

In [ ]:
def process_document(text):
    """
    Create a vector for given text and adjust it for cosine similarity search
    """
    text_vect = create_vector_from_text(tokenizer, model, text)
    text_vect = np.array(text_vect)
    text_vect = text_vect.reshape(1, -1)

    return text_vect


def is_plagiarism(similarity_score, plagiarism_threshold):

  is_plagiarism = False

  if(similarity_score >= plagiarism_threshold):
    is_plagiarism = True

  return is_plagiarism



def run_plagiarism_analysis(query_text, data, plagiarism_threshold=0.8):

    top_N=3


    # Preprocess the document to get the required vector for similarity analysis
    query_vect = process_document(query_text)

    # Run similarity Search
    data["similarity"] = data["vectors"].apply(lambda x: cosine_similarity(query_vect, x))
    data["similarity"] = data["similarity"].apply(lambda x: x[0][0])

    similar_articles = data.sort_values(by='similarity', ascending=False)[0:top_N+1]
    formated_result = similar_articles[["abstract", "paper_id", "similarity"]].reset_index(drop = True)

    similarity_score = formated_result.iloc[0]["similarity"]
    most_similar_article = formated_result.iloc[0]["abstract"]
    is_plagiarism_bool = is_plagiarism(similarity_score, plagiarism_threshold)

    plagiarism_decision = {'similarity_score': similarity_score,
                            'is_plagiarism': is_plagiarism_bool,
                            'most_similar_article': most_similar_article,
                            'article_submitted': query_text
                          }

    return plagiarism_decision

In [ ]:
new_incoming_text="This article is an updated version of the previous edition article by Charles M. Poser, volume 4, pp. 469–4814, © 2008, Elsevier Inc."

In [ ]:
text=vector_index['abstract'][88]

In [ ]:
text='''
Summary Background Current diagnostic tests are inadequate to detect typhoid cases, as well as the chronic carrier state, the sole reservoir of Salmonella enterica serovar Typhi. The current study was conducted to find new molecular signatures of pathogen/disease to understand the mechanism behind the host–pathogen interaction in enteric fever.'''

In [ ]:
print(text)


Summary Background Current diagnostic tests are inadequate to detect typhoid cases, as well as the chronic carrier state, the sole reservoir of Salmonella enterica serovar Typhi. The current study was conducted to find new molecular signatures of pathogen/disease to understand the mechanism behind the host–pathogen interaction in enteric fever.


In [ ]:
analysis_result = run_plagiarism_analysis(text, vector_index, plagiarism_threshold=0.8)

In [ ]:
analysis_result

{'similarity_score': 0.7469355,
 'is_plagiarism': False,
 'most_similar_article': "  We provide a new estimate of the local supermassive black hole mass function\nusing (i) the empirical relation between supermassive black hole mass and the\nSersic index of the host spheroidal stellar system and (ii) the measured\n(spheroid) Sersic indices drawn from 10k galaxies in the Millennium Galaxy\nCatalogue. The observational simplicity of our approach, and the direct\nmeasurements of the black hole predictor quantity, i.e. the Sersic index, for\nboth elliptical galaxies and the bulges of disc galaxies makes it\nstraightforward to estimate accurate black hole masses in early- and late-type\ngalaxies alike. We have parameterised the supermassive black hole mass function\nwith a Schechter function and find, at the low-mass end, a logarithmic slope\n(1+alpha) of ~0.7 for the full galaxy sample and ~1.0 for the early-type galaxy\nsample. Considering spheroidal stellar systems brighter than M_B = -1

#StatisticalBERT


In [ ]:
#already done in plagiarism
#!pip install transformers -q

In [ ]:
!pip install torch -q

In [ ]:
!pip install nltk -q

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel,TFBertTokenizer, AutoModelWithLMHead
import torch
import torch.nn.functional as F  # Import torch.nn.functional as F
import nltk
from nltk.util import ngrams
from nltk.lm.preprocessing import pad_sequence
from nltk.probability import FreqDist
import plotly.express as px
from collections import Counter
from nltk.corpus import stopwords
import string

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizerstat = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, return_attention_mask=False,
    return_token_type_ids=False)

Loading BERT tokenizer...


In [ ]:
statmodel = AutoModelWithLMHead.from_pretrained('bert-base-uncased')


In [ ]:

def calculate_perplexity(text, window_size=512, overlap=256):
    tokens = tokenizerstat.encode(text, add_special_tokens=True, return_tensors='pt')
    total_log_prob = 0.0
    total_tokens = 0

    for i in range(0, len(tokens[0]), window_size - overlap):
        start_idx = i
        end_idx = min(i + window_size, len(tokens[0]))

        chunk = tokens[:, start_idx:end_idx]

        with torch.no_grad():
            outputs = statmodel(chunk)
            logits = outputs.logits

        # Calculate cross entropy loss and accumulate it
        loss = F.cross_entropy(logits.view(-1, logits.size(-1)), chunk.view(-1), reduction='sum')
        total_log_prob += loss.item()
        total_tokens += len(chunk.view(-1))

    # Calculate average log probability and perplexity
    average_log_prob = total_log_prob / total_tokens
    perplexity = torch.exp(torch.tensor(average_log_prob))
    return perplexity.item()

def calculate_burstiness(text):
    tokens= nltk.word_tokenize(text.lower())
    word_freq= FreqDist(tokens)
    repeated_count =sum(count > 1 for count in word_freq.values())
    burstiness=repeated_count/ len(word_freq)
    return burstiness

In [ ]:
calculate_perplexity(x)

1.3390823602676392

In [ ]:
calculate_burstiness(x)

0.1702127659574468

# UI new Gradio

In [ ]:
!pip install https://github.com/amaiya/eli5-tf/archive/refs/heads/master.zip

  Using cached https://github.com/amaiya/eli5-tf/archive/refs/heads/master.zip
  Preparing metadata (setup.py) ... done


In [ ]:
!pip install pip install https://gradio-builds.s3.amazonaws.com/84b9651d1c6034283aadb332ec1db56543995837/gradio-4.16.0-py3-none-any.whl

  Using cached https://gradio-builds.s3.amazonaws.com/84b9651d1c6034283aadb332ec1db56543995837/gradio-4.16.0-py3-none-any.whl (16.5 MB)


In [ ]:
import gradio as gr

In [ ]:
%load_ext gradio

The gradio extension is already loaded. To reload it, use:
  %reload_ext gradio


In [ ]:
def analyse(query_text, data, plagiarism_threshold=0.8):

    top_N=3
    # Preprocess the document to get the required vector for similarity analysis
    query_vect = process_document(query_text)

     # Run similarity Search
    data["similarity"] = data["vectors"].apply(lambda x: cosine_similarity((query_vect), x))
    data["similarity"] = data["similarity"].apply(lambda x: x[0][0])


    similar_articles = data.sort_values(by='similarity', ascending=False)[0:top_N+1]
    formated_result = similar_articles[["abstract", "paper_id", "similarity"]].reset_index(drop = True)

    similarity_score = formated_result.iloc[0]["similarity"]
    most_similar_article = formated_result.iloc[0]["abstract"]
    is_plagiarism_bool = is_plagiarism(similarity_score, plagiarism_threshold)
    if is_plagiarism_bool == False and similarity_score<0.7:
      most_similar_article='None'
    return similarity_score, most_similar_article, is_plagiarism_bool

In [ ]:
def upload(boxdata, filedata):
    if not boxdata and not filedata:
      op=''
    elif boxdata:
      op= boxdata
      if len(op)<100:
        raise gr.Warning("Enter a longer prompt!")
    else:
      file = open(filedata,'r')
      fileread= file.read()
      op= fileread
    return op

In [ ]:
def test(datain):
    if not datain:
      raise gr.Error(("Please enter text or upload file!"))
    test=predictor.predict(datain)
    explained=predictor.explain(datain)
    perplexity= calculate_perplexity(datain)
    burstiness= calculate_burstiness(datain)
    score,artic,plagbool=analyse(datain, vector_index, plagiarism_threshold=0.8)
    return test,perplexity,burstiness,score,artic,plagbool,explained.data

In [ ]:
def upload_file(file):
    file_paths = file.name
    return file_paths

In [ ]:
%%blocks
with gr.Blocks(theme=gr.themes.Soft(radius_size="md")) as demo:
  gr.Markdown("""
    # AI Content Detection
    """)
  with gr.Row():
    with gr.Column():
      data = gr.Textbox(label="Text Input",placeholder="Enter your text here!",max_lines=7)
      file_output = gr.File(label="File Input")
      upload_button = gr.UploadButton("Click to Upload a File", file_types=["text"], file_count="single")
      upload_button.upload(upload_file, upload_button, file_output)
      with gr.Row():
          greet_btn = gr.Button("Check",variant="primary")
    with gr.Column():
      with gr.Row():
        pred = gr.Textbox(label="Prediction" ,max_lines=1)
        perp = gr.Textbox( label="Perplexity" ,max_lines=1)
        burs =gr.Textbox( label="Burstiness",max_lines=1)
        score= gr.Textbox( label= "Plagiarism Score",max_lines=1)
        plagbool=gr.Textbox( label= "Plagiarism Prediction",max_lines=1)
        artic=gr.Textbox( label= "Most Similar Article",max_lines=5)
  with gr.Row():

      gr.Markdown(""" # Analysis result
        Analysis shows expanation of prediction  """)
  explained= gr.HTML()
  gr.on([data.submit, greet_btn.click],
        fn=upload, inputs=[data,file_output], outputs=[data]
        ).then(test,inputs=data, outputs=[pred,perp, burs,score,artic,plagbool,explained])

  #greet_btn.click(fn=test, inputs=[data,file_output], outputs=[data,pred,perp, burs,score,artic,plagbool,explained])


<IPython.core.display.Javascript object>

#Highlighting

In [ ]:
%%blocks

with gr.Blocks() as demo:
    name = gr.Textbox(label="Name")
    output = gr.HTML()
    greet_btn = gr.Button("Greet")
    @greet_btn.click(inputs=name, outputs=output)
    def highlight_text(name):
      highlighted_text = "<mark>" + name + "</mark>"
      coloured_text="<span style='color:red'>" + name+ "</span>"
      return highlighted_text, coloured_text

<IPython.core.display.Javascript object>

In [ ]:
%%blocks
def highlight_text(name):
  val=[{'entity': '0.232',
  'start': 1,
  'end': 3},
 {  'start': 3,
  'end': 9},
 {'entity': 'LOC',
  'start': 11,
  'end': 20}]
  return {"text":name, "entities":val}

with gr.Blocks() as demo:
    name = gr.Textbox(label="Name")
    output = gr.Highlightedtext(label="try", combine_adjacent=True)
    butn = gr.Button()
    butn.click(fn= highlight_text, inputs=name, outputs=output )

demo.launch(debug=True, share=False)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
# Define a function to get sentence probabilities
def get_sentence_probabilities(sentence):
    # Use the predictor to predict the probability
    prob_ai_generated = predictor.predict_proba([sentence])[0]
    return {'probability_ai_generated': prob_ai_generated}

# Create a Gradio interface for input sentence and output probabilities
import gradio as gr

iface = gr.Interface(
    fn=get_sentence_probabilities,
    inputs=gr.Textbox(lines=5, label="Enter sentence"),
    outputs=gr.Label(num_top_classes=2, label='AI Generated Probability'),
    title="AI-Generated Sentence Detector",
    description="Enter a sentence to get the probability of it being AI-generated.",
)

iface.launch()

##TESTING PREDICTOR.PREDICT

In [ ]:
prob=predictor.predict_proba(x)
print(prob)
np.argmax(prob)

In [ ]:
!pip install https://github.com/amaiya/eli5-tf/archive/refs/heads/master.zip

In [ ]:
sentence='Once upon a time there was an old mother pig who had three little pigs and not enough food to feed them. So when they were old enough, she sent them out into the world to seek their fortunes. The first little pig was very lazy. He didnt want to work at all and he built his house out of straw. The second little pig worked a little bit harder but he was somewhat lazy too and he built his house out of sticks. Then, they sang and danced and played together the rest of the day. The third little pig worked hard all day and built his house with bricks. It was a sturdy house complete with a fine fireplace and chimney. It looked like it could withstand the strongest winds. The next day, a wolf happened to pass by the lane where the three little pigs lived; and he saw the straw house, and he smelled the pig inside. He thought the pig would make a mighty fine meal and his mouth began to water. So he knocked on the door and said:'
prob_ai_generated = predictor.explain(sentence, n_samples=2000)
prob_ai_generated

In [ ]:
sentence='Once upon a time there was an old mother pig who had three little pigs and not enough food to feed them. So when they were old enough, she sent them out into the world to seek their fortunes. The first little pig was very lazy. He didnt want to work at all and he built his house out of straw. The second little pig worked a little bit harder but he was somewhat lazy too and he built his house out of sticks. Then, they sang and danced and played together the rest of the day. The third little pig worked hard all day and built his house with bricks. It was a sturdy house complete with a fine fireplace and chimney. It looked like it could withstand the strongest winds. The next day, a wolf happened to pass by the lane where the three little pigs lived; and he saw the straw house, and he smelled the pig inside. He thought the pig would make a mighty fine meal and his mouth began to water. So he knocked on the door and said:'
prob_ai_generated = predictor.explain(sentence, n_samples=1900)
prob_ai_generated

In [ ]:
type()

In [ ]:
sentence='Once upon a time there was an old mother pig who had three little pigs and not enough food to feed them. So when they were old enough, she sent them out into the world to seek their fortunes. The first little pig was very lazy. He didnt want to work at all and he built his house out of straw. The second little pig worked a little bit harder but he was somewhat lazy too and he built his house out of sticks. Then, they sang and danced and played together the rest of the day. The third little pig worked hard all day and built his house with bricks. It was a sturdy house complete with a fine fireplace and chimney. It looked like it could withstand the strongest winds. The next day, a wolf happened to pass by the lane where the three little pigs lived; and he saw the straw house, and he smelled the pig inside. He thought the pig would make a mighty fine meal and his mouth began to water. So he knocked on the door and said:'
prob_ai_generated = predictor.explain(sentence, n_samples=1000)
prob_ai_generated

In [ ]:
sentence='Once upon a time there was an old mother pig who had three little pigs and not enough food to feed them. So when they were old enough, she sent them out into the world to seek their fortunes. The first little pig was very lazy. He didnt want to work at all and he built his house out of straw. The second little pig worked a little bit harder but he was somewhat lazy too and he built his house out of sticks. Then, they sang and danced and played together the rest of the day. The third little pig worked hard all day and built his house with bricks. It was a sturdy house complete with a fine fireplace and chimney. It looked like it could withstand the strongest winds. The next day, a wolf happened to pass by the lane where the three little pigs lived; and he saw the straw house, and he smelled the pig inside. He thought the pig would make a mighty fine meal and his mouth began to water. So he knocked on the door and said:'
prob_ai_generated = predictor.explain(sentence, n_samples=500)
prob_ai_generated

In [ ]:
prob_ai_generated = predictor.explain(x, n_samples=500)
prob_ai_generated

In [ ]:
prob_ai_generated = predictor.explain(x, n_samples=1000)
prob_ai_generated

In [ ]:
prob_ai_generated = predictor.explain(x, n_samples=1500)
prob_ai_generated

In [ ]:
prob_ai_generated = predictor.explain(x, n_samples=8000)
prob_ai_generated

In [ ]:
prob_ai_generated = predictor.explain(x, n_samples=1900, all_targets=True)
prob_ai_generated

In [ ]:
prob_ai_generated.metadata

In [ ]:
sentence='Once upon a time there was an old mother pig who had three little pigs and not enough food to feed them. So when they were old enough, she sent them out into the world to seek their fortunes. The first little pig was very lazy. He didnt want to work at all and he built his house out of straw. The second little pig worked a little bit harder but he was somewhat lazy too and he built his house out of sticks. Then, they sang and danced and played together the rest of the day. The third little pig worked hard all day and built his house with bricks. It was a sturdy house complete with a fine fireplace and chimney. It looked like it could withstand the strongest winds. The next day, a wolf happened to pass by the lane where the three little pigs lived; and he saw the straw house, and he smelled the pig inside. He thought the pig would make a mighty fine meal and his mouth began to water. So he knocked on the door and said:'
prob = predictor.predict_proba(sentence)
print(prob)


In [ ]:
sentence='Damn but they not enough food to feed them. So when they were old enough, she sent them out into the world to seek their fortunes. The first little pig was very lazy. He didnt want to work at all and he built his house out of straw. The second little pig worked a little bit harder but he was somewhat lazy too and he built his house out of sticks. Then, they sang and danced and played together the rest of the day. The third little pig worked hard all day and built his house with bricks. It was a sturdy house complete with a fine fireplace and chimney. It looked like it could withstand the strongest winds. The next day, a wolf happened to pass by the lane where the three little pigs lived; and he saw the straw house, and he smelled the pig inside. He thought the pig would make a mighty fine meal and his mouth began to water. So he knocked on the door and said:'
prob = predictor.predict_proba(sentence)
print(prob)
np.argmax(prob)

In [ ]:

prob = predictor.get_classes()
print(prob)

#SAVING OUTPUTS

In [ ]:
# save pipe.pkl to output data folder
!cp pipe.pkl /content/drive/MyDrive/Minor/Output/